In [ ]:
!pip install -U lightautoml
!pip install vininfo

In [2]:
# Standard python libraries
import logging
import os
import time
import requests
import re
logging.basicConfig(format='[%(asctime)s] (%(levelname)s): %(message)s', level=logging.INFO)

# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import torch
from vininfo import Vin

# Imports from our package
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task
import pandas_profiling

/kaggle/input/lightautomlcourse-hw1/sample_submission.csv

/kaggle/input/lightautomlcourse-hw1/train_data.csv

/kaggle/input/lightautomlcourse-hw1/test_data.csv

Parameters:

In [ ]:
N_THREADS = 4 # threads cnt for lgbm and linear models
N_FOLDS = 10 # folds cnt for AutoML # 5 больше фолдов
RANDOM_STATE = 42 # fixed random state for various reasons
TEST_SIZE = 0.2 # Test size for metric check
TIMEOUT = 3600 # Time in seconds for automl run
TARGET_NAME = 'final_price' # Target column name

In [ ]:
%%time

train_data = pd.read_csv('../input/lightautomlcourse-hw1/train_data.csv')
train_data.head()

In [ ]:
train_data.info()

In [ ]:
train_data['deal_type'].unique()

In [ ]:
test_data = pd.read_csv('../input/lightautomlcourse-hw1/test_data.csv')
# test_data.head()

In [ ]:
submission = pd.read_csv('../input/lightautomlcourse-hw1/sample_submission.csv')
# submission.head()

### from LightAutoML 2021 HW-1 Approach-1

In [ ]:
def vin_to_dic(vin_no):
    """Преобразует VIN-номер в словарь параметров автомобиля"""
    vin_dic = {}
    
    try:
        vin_info = Vin(vin_no)

        vin_dic['checksum_is_ok'] = vin_info.verify_checksum()
        vin_dic['country'] = vin_info.country
        vin_dic['manufacturer'] = vin_info.manufacturer
        vin_dic['region'] = vin_info.region
        vin_dic['produce_year'] = vin_info.years[0]
        vin_dic['model_year'] = vin_info.years[1]
        vin_dic['wmi'] = vin_info.wmi      # всемирный индекс изготовителя
        vin_dic['vds'] = vin_info.vds[:-1] # технические характеристики автомобиля 
        vin_dic['vis'] = vin_info.vis      # идентификационный номер автомобиля

        details = vin_info.details
        if details:
            vin_dic['details'] = True
            vin_dic['body'] = str(details.body)
            vin_dic['engine'] = str(details.engine)
            vin_dic['model'] = str(details.model)
            vin_dic['plant'] = str(details.plant)
            vin_dic['serial'] = str(details.serial)
            vin_dic['transmission'] = str(details.transmission)
        else:
            vin_dic['details'] = False
            
            for field in ['body', 'engine', 'model', 'plant', 'serial', 'transmission']:
                vin_dic[field] = None
    except:
        vin_dic['checksum_is_ok'] = False
    
    return vin_dic

Поиск особенностей данных

In [ ]:
# посмотрим на зависимость цены автомобиля определенной марки с примерно одинаковым пробегом
# от года выпуска
prius_data = train_data[(train_data.vehicle_manufacturer == 'TOYOTA') &
                        (train_data.vehicle_model == 'Prius') &
                        (train_data.current_mileage > 50000) &
                        (train_data.current_mileage < 100000)][['deal_type', 'vehicle_year', 'final_price']]
sns.scatterplot(data=prius_data, x='vehicle_year', y='final_price', hue='deal_type');

Вывод: нужно реализовывать две отдельные модели - для продажи и аренды (по признаку deal_type), так как цены выражены в разном масштабе

Чуть-чуть подсмотрим в тестовую выборку

... лишь убедимся, что есть марки автомобилей, которые не встречаются в обучающей выборке :-)

In [ ]:
train_manufacturers = set(train_data.vehicle_manufacturer.value_counts(sort=False).index.to_list())
test_manufacturers  = set(test_data.vehicle_manufacturer.value_counts(sort=False).index.to_list())

no_in_train = list(test_manufacturers - train_manufacturers)
print(f'{", ".join(no_in_train)} не встречались в обучающей выборке :-)')

### Удаление аномалий

#### Удаление лишних пробелов в строковых полях

In [ ]:
def dataset_strip_strings(data):
    """Удаление лишних пробелов в строковых полях набора данных"""
    for field in data.select_dtypes(include='object'):
        data[field] = data[field].str.strip()
    return data

In [ ]:
train_data = dataset_strip_strings(train_data)
test_data = dataset_strip_strings(test_data)

Удаление дубликатов
Удаление дубликатов не добавляет точности, возможно, из-за особенностей набора данных.

In [ ]:
#train_data = train_data.drop(index=train_data[train_data.drop(columns=['row_ID']).duplicated()].index)

#### Генерация новых признаков

Определение года набора данных для рассчета возраста автомобилей

In [ ]:
current_year = max(train_data['vehicle_year'].max(), test_data['vehicle_year'].max()) + 1
print('Dataset year:', current_year)

Объединение поля производителя и модели

In [ ]:
def concat_manufacturer_and_model(data):
    """Объединение поля производителя и модели"""
    data['vehicle_full_model'] = data['vehicle_manufacturer'] + data['vehicle_model'].fillna('')

In [ ]:
concat_manufacturer_and_model(train_data)
concat_manufacturer_and_model(test_data)

Разделение выборок для моделей продажи и аренды

In [ ]:
train_data.deal_type.unique()

In [ ]:
sale_sign = 'For Sale'

train_data_sale = train_data[train_data.deal_type == sale_sign]
train_data_rent = train_data[train_data.deal_type != sale_sign]

test_data_sale  = test_data[test_data.deal_type == sale_sign]
test_data_rent  = test_data[test_data.deal_type != sale_sign]

Подсчет количества продаж автомобиля по его VIN

In [ ]:
train_data_sale['vin_cum_count'] = 0
test_data_sale['vin_cum_count'] = 0

vin_cumcount_train = train_data_sale[~train_data_sale.car_vin.isna()].groupby('car_vin' )['car_vin'].cumcount() + 1
train_data_sale.loc[vin_cumcount_train.index, 'vin_cum_count'] = vin_cumcount_train

vin_cumcount_test = test_data_sale[~test_data_sale.car_vin.isna()].groupby('car_vin' )['car_vin'].cumcount() + 1
test_data_sale.loc[vin_cumcount_test.index, 'vin_cum_count'] = vin_cumcount_test

Набор данных содержит большое количество перепродаж без извлечения прибыли :-) Отмыв денег?

In [ ]:
train_data_sale[train_data_sale.vin_cum_count > 10]

In [ ]:
# посмотрим на перекупщиков
print('Обучение:')
display(train_data_sale[train_data_sale.car_vin == '4T1BD1FK5EU128824'][['vin_cum_count', 'vehicle_manufacturer', 'vehicle_model', 'car_vin', 'current_mileage', 'final_price']])
print('Тест:')
display(test_data_sale[test_data_sale.car_vin == '4T1BD1FK5EU128824'][['vin_cum_count', 'vehicle_manufacturer', 'vehicle_model', 'car_vin', 'current_mileage']])

Не будем реализовывать алгоритмическое решение для таких странных данных. Поручим это LightAutoML :-)

#### Признаки для модели продажи

In [ ]:
def fill_field_na(data, group_by_fields, field_name, default_value):
    """Заполнение пропусков в field_name по медиане с группировкой по полю group_by_fields"""
    data[field_name] = ( data.groupby(by=group_by_fields, dropna=False)[field_name]
                            .transform(lambda x: x.fillna(x.mode()).fillna(default_value)) )

In [ ]:
def create_sale_feats(data, train_data):
    data = data.copy()

    # is VIN unknown?
    data['car_vin_is_na'] = data['car_vin'].isna()
    
    # VIN parsing
    vin_data = data['car_vin'].apply(vin_to_dic).apply(pd.Series)
    data = data.join(vin_data)
    
    # округление года производства по VIN до целого
    data['produce_year'] = data['produce_year'].fillna(data['vehicle_year']).astype(int)

    # возраст автомобиля
    data['vehicle_age'] = current_year - data['produce_year']
    data['vehicle_age_1'] = 1 / data['vehicle_age']
    data['vehicle_age_1_log'] = 1 / np.log(data['vehicle_age'] + 1)
    
    # возраст модели
    data['model_year'] = data['model_year'].fillna(data['produce_year'] - data['vehicle_age']).astype(int)
    data['model_age'] = current_year - data['model_year']
    data['model_age_1'] = 1 / data['model_age']
    data['model_age_1_log'] = 1 / np.log(data['model_age'] + 1)
    # polynomial model age
    data['model_age_2'] = data['model_age'] ** 2
    
    # вычисление средней стоимости автомобиля по его VIN
    temp = train_data[~train_data.car_vin.isna()][['car_vin', 'final_price']]
    mean_prices = temp.groupby(['car_vin'])['final_price'].agg(['mean']).reset_index()
    mean_prices.columns = ['vehicle_full_model', 'mean_price']
    data = data.merge(mean_prices, how='left',on='vehicle_full_model')

    # вычисление средней стоимости автомобиля с группировкой по производителю и марке
    temp = train_data[['vehicle_full_model', 'final_price']]
    mean_prices = temp.groupby(['vehicle_full_model'])['final_price'].agg(['mean', 'max']).reset_index()
    mean_prices.columns = ['vehicle_full_model', 'mean_price1', 'max_price']
    data = data.merge(mean_prices, how='left',on='vehicle_full_model')
    data['mean_price'] = data['mean_price'].fillna(data['mean_price1'])
    data = data.drop(columns=['mean_price1'])

    # вычисление средней стоимости автомобиля с группировкой по производителю
    temp = train_data[['vehicle_manufacturer', 'final_price']]
    mean_prices = temp.groupby(['vehicle_manufacturer'])['final_price'].agg(['mean', 'max']).reset_index()
    mean_prices.columns = ['vehicle_manufacturer', 'mean_price1', 'max_price1']
    data = data.merge(mean_prices, how='left',on='vehicle_manufacturer')
    data['mean_price'] = data['mean_price'].fillna(data['mean_price1'])
    data['max_price'] = data['max_price'].fillna(data['max_price1'])
    data = data.drop(columns=['mean_price1', 'max_price1'])
    
    # произведение стоимостей на возраст
    data['mean_price_x_age'] = data['mean_price'] * data['vehicle_age']

    data['current_mileage_2'] = data['current_mileage'] ** 2

    data['vehicle_model_vds'] = data['vehicle_full_model'] + data['vds'].fillna('')

    data['luxury'] = data['vehicle_manufacturer'].isin([
                            'JAGUAR',
                            'MASERATI',
                            'FERRARI',
                            'BENTLEY',
                            'ROLLS-ROYCE',
                            'LAMBORGHINI'])
    
    # fillna
    group_by_fields = ['vehicle_manufacturer', 'vehicle_model']
    fill_field_na(data, group_by_fields, 'doors_cnt', '4/5')
    fill_field_na(data, group_by_fields, 'country', 'World')
    fill_field_na(data, group_by_fields, 'manufacturer', 'na')
    fill_field_na(data, group_by_fields, 'region', 'World')
    group_by_fields = ['vehicle_manufacturer', 'vehicle_model', 'vehicle_category', 'vehicle_year']
    fill_field_na(data, group_by_fields, 'wmi', 'na')
        
    return data

#### Признаки для модели аренды

In [ ]:
def create_rent_feats(data, train_data):
    data = data.copy()
    
    # возраст автомобиля
    data['vehicle_age'] = current_year - data['vehicle_year']
    data['vehicle_age_1'] = 1 / data['vehicle_age']
    data['vehicle_age_1_log'] = 1 / np.log(data['vehicle_age'] + 1)
        
    # вычисление средней стоимости автомобиля по его VIN
    temp = train_data[~train_data.car_vin.isna()][['car_vin', 'final_price']]
    mean_prices = temp.groupby(['car_vin'])['final_price'].agg(['mean']).reset_index()
    mean_prices.columns = ['vehicle_full_model', 'mean_price']
    data = data.merge(mean_prices, how='left',on='vehicle_full_model')

    # вычисление средней стоимости автомобиля с группировкой по производителю и марке
    temp = train_data[['vehicle_full_model', 'final_price']]
    mean_prices = temp.groupby(['vehicle_full_model'])['final_price'].agg(['mean', 'max']).reset_index()
    mean_prices.columns = ['vehicle_full_model', 'mean_price1', 'max_price']
    data = data.merge(mean_prices, how='left',on='vehicle_full_model')
    data['mean_price'] = data['mean_price'].fillna(data['mean_price1'])
    data = data.drop(columns=['mean_price1'])

    # вычисление средней стоимости автомобиля с группировкой по производителю
    temp = train_data[['vehicle_manufacturer', 'final_price']]
    mean_prices = temp.groupby(['vehicle_manufacturer'])['final_price'].agg(['mean', 'max']).reset_index()
    mean_prices.columns = ['vehicle_manufacturer', 'mean_price1', 'max_price1']
    data = data.merge(mean_prices, how='left',on='vehicle_manufacturer')
    data['mean_price'] = data['mean_price'].fillna(data['mean_price1'])
    data['max_price'] = data['max_price'].fillna(data['max_price1'])
    data = data.drop(columns=['mean_price1', 'max_price1'])
    
    # произведение стоимостей на возраст
    data['mean_price_x_age'] = data['mean_price'] * data['vehicle_age']
    
    data['current_mileage_2'] = data['current_mileage'] ** 2
    
    return data

#### Функции для обучения

In [ ]:
def run_model(train_data, test_data, drop_fields):
    """Обучает LightAutoML на train_data, отбрасывая список признаков из drop_fields.
    Возвращает ошибку на train_data и предсказания для test_data."""
    task = Task('reg', loss='mae', metric = 'mae')
    
    roles = {
        'target': TARGET_NAME,
        'drop': drop_fields,
    }

    #automl = TabularAutoML(
    automl = TabularUtilizedAutoML(
                task = task,
                timeout = TIMEOUT,
                cpu_limit = N_THREADS,
                general_params = {
                    'nested_cv': False,
                    'use_algos': [
                        #['lgb', 'lgb_tuned', 'linear_l2', 'cb', 'cb_tuned'],
                        #['lgb', 'linear_l2'],
                        #['lgb', 'lgb_tuned', 'cb', 'cb_tuned'],
                        #['linear_l2'],
                        #['cb'],
                        ['linear_l2', 'lgb', 'lgb_tuned'],
                    ]
                },
                reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE},
            )

    oof_pred = automl.fit_predict(train_data, roles = roles)
    error_value = mean_absolute_error(train_data[TARGET_NAME].values, oof_pred.data[:, 0])
    print(f'Ошибка на out-of-fold выборке: {error_value}')

    try:
        fast_fi = automl.get_feature_scores('fast')
        fast_fi.set_index('Feature')['Importance'].plot.bar(figsize = (16, 10), grid = True)
        plt.show()
    except:
        print('Ошибка построения диаграммы важности признаков.')

    test_pred = automl.predict(test_data)

    return  error_value, test_pred

#### Обучение модели продажи автомобилей

In [ ]:
sale_error, sale_pred = run_model(create_sale_feats(train_data_sale, train_data_sale),
                                  create_sale_feats(test_data_sale, train_data_sale),
                                  ['row_ID',
                                   'vis'])

#### Обучение модели аренды автомобилей

In [ ]:
rent_error, rent_pred = run_model(create_rent_feats(train_data_rent, train_data_rent),
                                  create_rent_feats(test_data_rent, train_data_rent),
                                  ['row_ID'])

In [ ]:
submission.loc[test_data_sale.index, TARGET_NAME] = sale_pred.data[:, 0]
submission.loc[test_data_rent.index, TARGET_NAME] = rent_pred.data[:, 0]
submission.head()

## 5. Предварительный анализ

In [ ]:
train_data.profile_report()

In [ ]:
test_data.profile_report()

## 6. Сплит на sale и rent

Some user feature preparation

In [ ]:
train_data_sale = train_data[train_data['deal_type']=='For Sale']
test_data_sale = test_data[test_data['deal_type']=='For Sale']

train_data_rent = train_data[train_data['deal_type']=='For Rent']
test_data_rent = test_data[test_data['deal_type']=='For Rent']

## 7. Feature engineering

In [ ]:
dict_country_manuf = {
    'MAZDA' : 'Japan',
    'CHRYSLER' : 'USA',
    'HUMMER' : 'USA',
    'SAAB' : 'Sweden',
    'UAZ' : 'Russia',
    'MASERATI' : 'Italy',
    'TESLA' : 'USA',
    'MG' : 'Great Britain',
    'HYUNDAI' : 'South Korea',
    'LINCOLN' : 'USA',
    'PONTIAC' : 'USA',
    'LEXUS' : 'Japan',
    'OTHER' : 'Other',
    'AUDI' : 'Germany',
    'MERCURY' : 'USA',
    'TOYOTA' : 'Japan',
    'NISSAN' : 'Japan',
    'SKODA' : 'Czech Republic',
    'PORSCHE' : 'Germany',
    'GREATWALL' : 'China',
    'SUZUKI' : 'Japan',
    'BENTLEY' : 'Great Britain',
    'FOTON' : 'China',
    'SSANGYONG' : 'South Korea',
    'FIAT' : 'Italy',
    'MITSUBISHI' : 'Japan',
    'ALFA ROMEO' : 'Italy',
    'PEUGEOT' : 'France',
    'KIA' : 'South Korea',
    'HAVAL' : 'China',
    'VAZ' : 'Russia',
    'SUBARU' : 'Japan',
    'FERRARI' : 'Italy',
    'BMW' : 'Germany',
    'JEEP' : 'USA',
    'JAGUAR' : 'Great Britain',
    'ISUZU' : 'Japan',
    'RENAULT' : 'France',
    'FORD' : 'USA',
    'MOSKVICH' : 'Russia',
    'CADILLAC' : 'USA',
    'VOLVO' : 'Sweden',
    'INFINITI' : 'Japan',
    'HONDA' : 'Japan',
    'ASTON MARTIN' : 'Great Britain',
    'BUICK' : 'USA',
    'TATA' : 'India',
    'SEAT' : 'Spain',
    'MERCEDES-BENZ' : 'Germany',
    'DAEWOO' : 'South Korea',
    'LAND ROVER' : 'Great Britain',
    'VOLKSWAGEN' : 'Germany',
    'OPEL' : 'Germany',
    'ROVER' : 'Great Britain',
    'ROLLS-ROYCE' : 'Great Britain',
    'DODGE' : 'USA',
    'JAC' : 'China',
    'CHEVROLET' : 'USA',
    'SATURN' : 'USA',
    'SCION' : 'Japan',
    'LAMBORGHINI' : 'Italy',
    'GAZ' : 'Russia',
    'ACURA' : 'Japan',
    'CITROEN' : 'France',
    'MINI' : 'Great Britain',
    'ZAZ' : 'Russia',
    'CHERY' : 'China',
    'GMC' : 'USA',
    'DAIHATSU' : 'Japan',
    'LANCIA' : 'Italy'
}

In [ ]:
dict_level_price_country ={
    'China' : 6,
    'Czech Republic' : 7,
    'France' : 9,
    'Germany' : 14,
    'Great Britain' : 18,
    'India' : 1,
    'Italy' : 13,
    'Japan' : 20,
    'Other' : 0,
    'Russia' : 4,
    'South Korea' : 12,
    'Spain' : 8,
    'Sweden' : 10,
    'USA' : 15
}

In [ ]:
sorted(list(dict_level_price_country.values()))

In [ ]:
dict_sport_car_country = {
    'MASERATI' : '1',
    'ALFA ROMEO' : '1',
    'FERRARI' : '1',
    'ASTON MARTIN' : '1',
    'LAMBORGHINI' : '1',
}

In [ ]:
dict_wheels = {'  Right-hand drive   ':0, 
                    '  Left wheel  ':1}

In [ ]:
dict_vehicle_color = {
    ' Silver ' : 5,
    ' Blue ' : 14,
    ' Black ' : 20,
    ' White ' : 1,
    ' Grey ' : 16,
    ' Red ' : 8,
    ' Brown ' : 18,
    ' Green ' : 12,
    ' Carnelian red ' : 9,
    ' Beige ' : 2,
    ' Sky blue ' : 11,
    ' Golden ' : 7,
    ' Yellow ' : 3,
    ' Purple ' : 10,
    ' Orange ' : 6,
    ' Pink ': 4
}

In [ ]:
sorted(list(dict_vehicle_color.values()))

In [ ]:
dict_extra_vehicle_color = {
    ' Green '  : 1,
    ' Golden ' : 1,
    ' Yellow ' : 1,
    ' Purple ' : 1,
    ' Orange ' : 1,
    ' Pink '   : 1
}

In [ ]:
set_manuf_train = set(train_data.vehicle_manufacturer.unique())
set_manuf_test = set(test_data.vehicle_manufacturer.unique())
list_manuf_to_drop = list(set_manuf_train-set_manuf_test)
print(list_manuf_to_drop)

In [ ]:
list_manuf_no_in_train = list(set_manuf_test - set_manuf_train)
print(list_manuf_no_in_train)

In [ ]:
dict_manuf_count = train_data.vehicle_manufacturer.value_counts().to_dict()
dict_manuf_count['OTHER'] = dict_manuf_count['სხვა']

In [ ]:
dict_manuf_count_test = test_data.vehicle_manufacturer.value_counts().to_dict()
dict_manuf_count['CHERY'] = dict_manuf_count_test['CHERY']
dict_manuf_count['HAVAL'] = dict_manuf_count_test['HAVAL']

In [ ]:
df_group_manuf_mean_price = train_data.groupby('vehicle_manufacturer').final_price.mean()
border_budget, border_medium, border_expensive = tuple(df_group_manuf_mean_price.quantile([0.25,0.5,0.75]).to_list())
border_budget, border_medium, border_expensive

In [ ]:
def type_price_manuf(row):
    if row <= border_budget:
        return 1
    if row <= border_medium:
        return 2
    if row <= border_expensive:
        return 3
    else:
        return 4
dict_manuf_type_price = df_group_manuf_mean_price.apply(type_price_manuf).to_dict()
dict_manuf_type_price['OTHER'] = dict_manuf_type_price['სხვა']

In [ ]:
dict_manuf_type_price['CHERY'] = 2
dict_manuf_type_price['HAVAL'] = 2

In [ ]:
%%time
def cat_vehicle_year_2(row):
    if row <= 3:
        return 1
    if row <= 5:
        return 2
    if row <= 7:
        return 3
    if row <= 10:
        return 4
    if row <= 20:
        return 5
    # if row <= 25:
    #     return 6
    else:
        return 6

def create_expert_feats(data):
    
    # замена грузинского слова на английское
    dict_other_manuf = {'სხვა':'OTHER'}
    data.vehicle_manufacturer = data.vehicle_manufacturer.apply(lambda row: dict_other_manuf[row] if row in dict_other_manuf.keys() else row)
    
    # популярность производителя
    data['manuf_count'] = data.vehicle_manufacturer.apply(lambda row: int(dict_manuf_count[row]) if row in dict_manuf_count.keys() else np.nan)
    
    # четыре типа цен по производителям (бюджетный, средний, дорогой, очень дорогой)
    data['manuf_type_price'] = data.vehicle_manufacturer.apply(lambda row: int(dict_manuf_type_price[row]) if row in dict_manuf_type_price.keys() else np.nan)
    
#     дроп производителей которых нет в тесте
    data = data[~train_data['vehicle_manufacturer'].isin(list_manuf_to_drop)]

#     страна производителя
    data['country_manuf'] = data.vehicle_manufacturer.apply(lambda row: dict_country_manuf[row] if row in dict_country_manuf.keys() else np.nan)
    
    # уровень цен от страны производителя 
    data['level_price_country'] = data.country_manuf.apply(lambda row: int(dict_level_price_country[row]) if row in dict_level_price_country.keys() else np.nan)

#     спорткары по производителю 
    data['sport_car_country'] = data.country_manuf.apply(lambda row: dict_sport_car_country[row] if row in dict_sport_car_country.keys() else 0)

    # дроп названия страны после использования
    data.drop(['country_manuf'], axis=1, inplace=True)

    # инверт даты в логнормальн. распред
    data['vehicle_year_2'] = 2021 - data.vehicle_year
    
    # интенсивность использования авто
    data['intensity_use'] = data.current_mileage/data.vehicle_year_2

    # категоризация возраста авто по группам
    data['cat_vehicle_year_2'] = data.vehicle_year_2.apply(cat_vehicle_year_2)
    
    # руль в бинарный признак с заменой на популярный
    data['wheels_2'] = data.wheels.apply(lambda row: dict_wheels[row] if row in dict_wheels.keys() else 1)

    # новые признаки константы и nan
    data['constant'] = 1
    data['allnan'] = np.nan
    
    # градация цвета от светлого к темному
    data['vehicle_color_2'] = data.vehicle_color.apply(lambda row: dict_vehicle_color[row] if row in dict_vehicle_color.keys() else np.nan)

    # редкие цвета
    data['extra_vehicle_color'] = data.vehicle_color.apply(lambda row: dict_extra_vehicle_color[row] if row in dict_extra_vehicle_color.keys() else 0)

    return
create_expert_feats(train_data)
create_expert_feats(test_data)

create_expert_feats(train_data_sale)
create_expert_feats(test_data_sale)

In [ ]:
# дроп одной строки из трейна
index_drop = train_data_sale[train_data_sale['vehicle_year']==0].index
train_data_sale.drop(index_drop, inplace=True)

index_drop = train_data[train_data['vehicle_year']==0].index
train_data.drop(index_drop, inplace=True)

In [ ]:
train_data_sale.head()

In [ ]:
test_data_sale.head()

### Свои фичи

In [ ]:
def new_feature_year_from(data):
    year_from = 2021- data['vehicle_year']
    data['year_from'] = year_from
def new_feature_cur_millage_divided(data):
    curr_millage_div = data['current_mileage'].apply(lambda x: int(x/10000))
    data['cur_millage_divided'] = curr_millage_div
    
def new_feature_man_model(data):
    man_model = data['vehicle_manufacturer'] + ' ' + data['vehicle_model']
    data['man_model'] = man_model

new_feature_year_from(train_data)
new_feature_year_from(test_data)

new_feature_cur_millage_divided(train_data)
new_feature_cur_millage_divided(test_data)

new_feature_man_model(train_data)
new_feature_man_model(test_data)

In [ ]:
def get_model(name):
    if type(name)==str:
        name=name.lower()
        model_search = re.search('(\d\d+)', name)
        if model_search:
            return model_search.group(1)
    return name

def get_model2(name):
    if type(name)==str:
        model_search = re.search('([\w\-]+)', name)
        if model_search:
            return model_search.group(1)
    return name

def create_expert_feats(data):
    data['current_mileage_over'] = data['current_mileage'].map(lambda x: False if x<750000 else True)
 
    data['vehicle_model2'] = data['vehicle_model'].apply(get_model)
    data['vehicle_model2'] = data['vehicle_model2'].apply(get_model2)
    
    data['has_vin'] = data["car_vin"].map(lambda x: 1 - int(type(x) == float))
    data["car_vin"] = data["car_vin"].map(lambda x: x if (type(x) != float and \
                      len(x)==19 and 'O' not in x and 'Q' not in x and 'I' not in x and \
                      Vin(x).verify_checksum()) else np.nan)
    
    data['vin_vds'] = data["car_vin"].map(lambda x: Vin(x).vds[0:5] if (type(x) != float) else x)
    data['vin_wmi'] = data["car_vin"].map(lambda x: Vin(x).wmi if (type(x) != float) else x)
    data['vin_wmi_vds'] = data['vin_wmi'] + data['vin_vds']
    data['vin_country'] = data["car_vin"].map(lambda x: Vin(x).country if (type(x) != float) else x)
    data['vin_region'] = data["car_vin"].map(lambda x: Vin(x).region if (type(x) != float) else x)
    
    data['used_years'] = 2021 - data['vehicle_year']

    data['mileage_at_year'] = data['current_mileage'] / data['used_years']

create_expert_feats(train_data)
create_expert_feats(test_data)

In [ ]:
def create_extra_features(data):
    data['NANs_cnt'] = data.isnull().sum(axis = 1) 
    
def create_col_with_min_freq(data, col, min_freq = 10):
    # replace rare values (less than min_freq rows) in feature by RARE_VALUE
    data[col + '_fixed'] = data[col].astype(str)
    data.loc[data[col + '_fixed'].value_counts()[data[col + '_fixed']].values < min_freq, col + '_fixed'] = "RARE_VALUE"
    data.replace({'nan': np.nan}, inplace = True)

def create_gr_feats(data):
    # create aggregation feats for numeric features based on categorical ones
    for cat_col in ['vehicle_manufacturer', 'vehicle_model', 'vehicle_category',
                   'vehicle_gearbox_type', 'doors_cnt', 'wheels', 'vehicle_color', 
                   'vehicle_interior_color', 'deal_type']:
        create_col_with_min_freq(data, cat_col, 15)
        for num_col in ['current_mileage', 'vehicle_year', 'car_leather_interior']:
            for n, f in [('mean', np.mean), ('min', np.nanmin), ('max', np.nanmax)]:
                data['FIXED_' + n + '_' + num_col + '_by_' + cat_col] = data.groupby(cat_col + '_fixed')[num_col].transform(f)
                
    # create features with counts
    for col in ['vehicle_manufacturer', 'vehicle_model', 'vehicle_category',
               'current_mileage', 'vehicle_year', 'vehicle_gearbox_type', 'doors_cnt',
               'wheels', 'vehicle_color', 'vehicle_interior_color', 'car_vin', 'deal_type']:
        data[col + '_cnt'] = data[col].map(data[col].value_counts(dropna = False))
    
        

create_extra_features(train_data)
create_extra_features(test_data)

all_df = pd.concat([train_data, test_data]).reset_index(drop = True)
create_gr_feats(all_df)
train_data, test_data = all_df[:len(train_data)], all_df[len(train_data):]
print(train_data.shape, test_data.shape)

In [ ]:
train_data.head()

In [ ]:
# train_data['cur_millage_divided'].unique()

In [ ]:
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt

cat_cols = ['vehicle_manufacturer', 'man_model', 'vehicle_category', 'vehicle_gearbox_type', 'doors_cnt', 'wheels','vehicle_color',
            'vehicle_interior_color','deal_type']

# copy
# df = train_data.copy()
# encoder = LabelEncoder()
# df[cat_cols] = df[cat_cols].apply(encoder.fit_transform)

In [ ]:
# df.info()

In [ ]:
# df.head(5)

In [ ]:
# корр матрица
# corrMatrix = df.corr()
# sns.heatmap(corrMatrix, annot=True)
# plt.rcParams["figure.figsize"] = (40,10)

In [ ]:
# sns.pairplot(df, hue="final_price")

Create Task

In [ ]:
task = Task('reg', loss='mae', metric='mae')

 Setup columns roles

In [ ]:
roles = {'target': TARGET_NAME,
         'drop': ['row_ID'] # to drop or not to drop?
         ,'category': ['vehicle_manufacturer', 'vehicle_model', 'vehicle_category', 'vehicle_year', 'doors_cnt', 'wheels',
                 'vehicle_color', 'vehicle_interior_color'] #вручную установим категории
         }
# 'car_vin', 'vehicle_manufacturer', 'vehicle_model', 'current_mileage', 'vehicle_year'

In [ ]:
%%time 

automl = TabularUtilizedAutoML(task = task, 
                       timeout = TIMEOUT,
                       cpu_limit = N_THREADS,
                       general_params = {'use_algos': [['linear_l2', 'lgb', 'lgb_tuned']]},
                       reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE},
#                        general_params = {'use_algos': [['cb']]}
                      )
oof_pred = automl.fit_predict(train_data, roles = roles)
logging.info('oof_pred:\n{}\nShape = {}'.format(oof_pred, oof_pred.shape))

Create AutoML from preset

In [ ]:
# Fast feature importances calculation
fast_fi = automl.get_feature_scores('fast')
fast_fi.set_index('Feature')['Importance'].plot.bar(figsize = (20, 10), grid = True)

 Predict to test data and check scores

In [ ]:
%%time

test_pred = automl.predict(test_data)
logging.info('Prediction for test data:\n{}\nShape = {}'
              .format(test_pred, test_pred.shape))

logging.info('Check scores...')
logging.info('OOF score: {}'.format(mean_absolute_error(train_data[TARGET_NAME].values, oof_pred.data[:, 0])))

Generate submission

In [ ]:
submission[TARGET_NAME] = test_pred.data[:, 0]
submission.head()

In [3]:
submission.to_csv('lightautoml_sample_submission.csv', index = False)

NameError: name 'submission' is not defined